In [ ]:
#!/usr/bin/python

'''
File name : OpenRetina_manipulations.ipynb
Authors : Pierre Albiges, Victor Boutin and Laurent Perrinet
Date created : 2017-04-25
Date last modified : 2017-04-25
Object : Apprehend the OpenRetina project through the manipulation of the Photoreceptor class

See https://github.com/laurentperrinet/openRetina/blob/master/src/openRetina.py
'''

from __future__ import division, print_function
import io
import struct
import array
import numpy as np
import zmq
import time
import sys

from multiprocessing.pool import ThreadPool
from collections import deque

class PhotoReceptor :
    def __init__(self, w, h, cam_id=0, DOWNSCALE=1, verbose = True) :
        self.sleep_time = 2 #Letting the camera warming up for 2 seconds
        self.fps = 90
        self.led = False
        print ('h = ', h, 'w = ', w)
        
        '''Which camera handler will be used?'''
        try :
            '''On Raspbian'''
            import picamera
            import picamera.array
            
            self.rpi = True #What rpi means and does?
            self.cap = picamera.PiCamera(cam_id)
            self.cap.start_preview()
            
            if DOWNSCALE > 1 :
                if verbose : #Only 'if verbose' ?
                    print('DOWNSCALE not implemented yet on the Pi')
            
            self.cap.resolution = (self.w, self.h)
            self.cap.framerate = self.fps
            time.sleep(self.sleep_time)
            
            with picamera.array.PiRGBArray(self.cap) as self.stream:
                self.cap.capture(self.stream, format='rgb')
            
        except :
            '''On other Unix systems'''
            self.rpi = False
            
            try :
                import cv2
                self.cap = cv2.VideoCapture(cam_id)
                if not self.cap.isOpened() :
                    toto # What does it do?
                
                print ("dim1 : {0}, dim2 : {1}".format(self.h,self.w)) #I don't understand that line
                self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.w)
                self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.h)
                
                self.DOWNSCALE = DOWNSCALE
                if DOWNSCALE > 1 :
                    W = self.cap.get(cv2.CAP_PROP_FRAME_WIDTH)
                    H = self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT)
                    self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, W/self.DOWNSCALE)
                    self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H/self.DOWNSCALE)
                self.h, self.w = self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT), self.cap.get(cv2.CAP_PROP_FRAME_WIDTH)
                if verbose:
                    print ('Using OpenCV') #Again, only 'if verbose'?
                    
            except :
                if verbose :
                    print ('Unable to capture video')
                    
    def grab(self) :
        if self.rpi : #At this point, the image is available as stream.array
            frame = self.stream.array
        else :
            ret, frame_bgr = self.cap.read() #What does ret and bgr means?
            frame = frame_bgr[:, :, ::-1] #BGR to RBG
        return frame
    
    def close(self) :
        if self.rpi : #is == True ?
            self.camera.stop_preview()
            self.camera.close()
        else :
            self.cap.release()
            del self.cap

Try = PhotoReceptor()

In [ ]:
help(Try)